<font color="#de3023"><h1><b>REMINDER: MAKE A COPY OF THIS NOTEBOOK, DO NOT EDIT</b></h1></font>

To copy the notebook, go to File and click create "Save a copy to ..." and work on that copy.

Please submit a pdf of the page of your notebook (Ctrl + p on the page, save as pdf, and submit that pdf) on gradescope.

Please remember to assign pages to the appropriate questions. Not doing so will result in the deduction of points. Please submit a **pdf** version of the colab notebook.

We will not rerun your uploaded notebook, so make sure to run each cell before downloading so that all outputs and plots are visible on the saved pdf.


Homework 5: In this homework, you will train a two layer neural network using gradient descent. However, instead of manually computing the gradients, you will use the autodiff provided by Tensorflow package.

## Question 0 Part 1
Do you have confusions or questions about the previous lectures?  (This is optional to answer)

(Answer here)

## Question 0 Part 2
Any suggestions or thoughts about the course? (This is optional to answer)

(Answer here)

In [ ]:
import numpy as np
import tensorflow as tf

# Two-layer Neural Network


A two layer neural network contains an input layer, a hidden layer, and an output layer. The number of the input nodes in the diagram below is determined the dimension of our features. We are free to choose the dimension of the hidden layer. As for the final output layer, the number of nodes is determined by the type of the problem we are doing. For instance, for a regression problem, we will only have one node and the output value corresponds to our prediction of the target. As for classification, we will first output a vector that has same number of dimension as the number of classes in our classification dataset. Then, we will apply the softmax transformation on the vector to transform real-valued predictions to the class probabilities.
![](https://www.researchgate.net/profile/Haohan-Wang-4/publication/282997080/figure/fig4/AS:305939199610886@1449952997594/A-typical-two-layer-neural-network-Input-layer-does-not-count-as-the-number-of-layers-of.png)

Mathematically, this model can be written as

$$f(x) =  \sigma(x^{\intercal} W_1  +b_1) W_2 + b_2. $$

Note that if $x \in \mathbb{R}^{d \times 1}$, then $W_1 \in \mathbb{R}^{d \times H}$ and $W_2 \in \mathbb{R}^{H \times O}$, where $O$ is the output dimension. The dimension of $b_1$ and $b_2$ is self-evident.

Given an input $x$, the model first transforms it using the weight matrix $W_1$ and subsequently shifts the output by the bias term $b_1$. The function $\sigma(.)$ is called the activation function that introdues non-linearity in the model. For the purpose of this homework, we will use the so called relu-activation function that is defined as $\sigma(t) = \text{max}\{t, 0\}$. Note that that $x^{\intercal} W_1  +b_1$ generally gives us a vector, so we have to apply the relu activation to each element of the vector. Following the activation, the vector $h(x) =\sigma(x^{\intercal} W_1  +b_1)$ is defines the hidden layer. Finally, we apply the linear trasformation $ h(x) W_1 + b_2$ on the hidden layer.



This representation of the network is very convenient if you instead want to do matrix operations on your data. Suppose $X$ be your data matrix where $i^{th}$ row of $X$ contains $x_i^{\intercal}$, then the output of the network on the entire dataset can be written as
$$f(X) = \sigma(X W_1  +b_1) W_2 + b_2. $$

In the previous HW, you trained a linear regression model on california housing dataset. In this homework, you will train a two-layer neural network on this dataset using gradient descent.

In [ ]:
# load the dataset
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

california_housing = fetch_california_housing( return_X_y=True, as_frame=True)
X = california_housing[0]
y = california_housing[1]
X_train_unscaled, X_test_unscaled, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
sc=StandardScaler()
X_train=sc.fit_transform(X_train_unscaled)
X_test = sc.transform(X_test_unscaled)

In [ ]:
# convert numpy arrays to tensors

X_train = tf.convert_to_tensor(X_train, dtype = tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype = tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype = tf.float32)
y_test= tf.convert_to_tensor(y_test, dtype = tf.float32)

<font color="#de3023"><h6><b>Question 1: Fill in the input dimension and output dimension of the two layer neural network for regression on this dataset. (5 pts) </b></h6></font>
For the purpose of this homework, we will just choose hidden layer with dimension double that of input dimension. However, going forward choosing the hidden dimension appropriately would be an important part of deep learning.

In [ ]:
# replace ______ with your code
input_dim = X_train.shape[1]
# print(input_dim)
hidden_dim = 2 * input_dim
# print(hidden_dim)
output_dim = 1
# print(output_dim)

<font color="#de3023"><h6><b>Question 2: Define a tensorflow variables for weights W1, b1, W2, and b2. Then, initialize both biases b1 and b2 to be 0 vectors and initialize W1 and W2 by picking values uniformly at random from the interval [0,1]. (10 pts) </b></h6></font>

In [ ]:
# replace ______ with your code
W1 = tf.Variable(tf.random.uniform(shape=(input_dim, hidden_dim)), trainable=True)
b1 = tf.Variable(tf.zeros(hidden_dim))
W2 = tf.Variable(tf.random.uniform(shape=(hidden_dim, output_dim)), trainable=True)
b2 = tf.Variable(tf.zeros(output_dim))

In [ ]:
b1.shape

TensorShape([16])

<font color="#de3023"><h6><b>Question 3: Complete the model function below to define a two layer neural network. Here, inputs is a matrix of shape $n \times d$, where $i^{th}$ row of the inputs matrix contains $x_i^{\intercal}$. (15 pts) </b></h6></font>
Hint: Use [tf.nn.relu()](https://www.tensorflow.org/api_docs/python/tf/nn/relu) function for relu activation.

In [ ]:
# Replace "________" with your code
def model(inputs):
  # print("model")
  hidden = tf.nn.relu(tf.add(tf.matmul(inputs, W1), b1))    # hidden layer with relu-activation
  # print("model2")
  # print("Hidden shape : ", hidden.shape)
  # print("W2 shape : ", W2.shape)
  output = hidden @ W2 + b2            # fully-connected output linear layer
  # print("Output shape : ", output.shape)
  # print("model end")
  return output

<font color="#de3023"><h6><b>Question 4: Write a function below that computes mean-squared error given the predictions and targets. You should use tensorflow operations like tf.square and tf.reduce_mean for autodiff to work.  (15 pts) </b></h6></font>

In [ ]:
# Replace "________" with your code
def mse(predictions, targets):
  # print("mse")
  # print(predictions.shape)
  # print(targets.shape)
  squared_error = tf.square(tf.subtract(predictions, targets))
  # print("Mse2")
  mean_squared_error = tf.reduce_mean(squared_error)
  # print("mse end")
  return mean_squared_error

<font color="#de3023"><h6><b>Question 5: Complete the function below that takes in features and targets and trains your two layer neural network using gradient descent. Please use autodiff by Gradient taping.  (25 pts) </b></h6></font>

Hint: This pseudocode should give you an essence of what is expected from your solution
```
learning_rate = 0.01
def train(inputs, targets):
  start your tape: #You need to write a statement here that intializes a gradient tape
    make predictions using model function in question 3
    observe loss of your predictions using mse function in question 4
  grad_w1, grad_b1, grad_w2, grad_b2 = tape the gradient of both weights and biases
  w1 = w1 - learning_rate*grad_w1 # use the appropriate tf functions for this
  Do the above step for w2, b1, b2 with its respective gradients

    
```




In [ ]:
# Replace "________" with your code
learning_rate = 0.01

def train(inputs, targets):
    with tf.GradientTape() as tape:
      # print("train")
      predictions = model(inputs)           #get predictions
      loss = mse(predictions, targets)           #compute loss
    dW1, db1, dW2, db2 = tape.gradient(loss, [W1, b1, W2, b2])        #tape the gradients of both weights and biases

    #update all weights and biases using the gradients computed above
    W1.assign_sub(dW1 * learning_rate)
    b1.assign_sub(db1 * learning_rate)
    W2.assign_sub(dW2 * learning_rate)
    b2.assign_sub(db2 * learning_rate)


<font color="#de3023"><h6><b>Question 6: Complete the routine below that divides the randomly shuffled data into multiple minibatches of size 500 and use the train function above to run gradient descent on those minibatches. Within each step, you should make a complete pass through the dataset. (25 pts) </b></h6></font>

Hint: This pseudocode should give you an essence of what is expected from your solution
```
k = number of minibatches of size 500
for step in range(100):
  X = ...
  y = ...
  for i in range(k): #for the minibatch iterations
    run the train method on your ith minibatch
  run the train method on the last minibatch with less than 500 samples
  compute loss
    
```

In [113]:
B = 1000
n = X_train.shape[0]
k = int(n/B)
for step in range(100):
    X_train = tf.random.shuffle(X_train, seed = step)   #separately shuffling X_train and y_train is reasonable here because of the same seed
    y_train = tf.random.shuffle(y_train, seed = step)
    xbatches = [X_train[j * B:(j + 1)*B] for j in range(k)]
    ybatches = [y_train[j * B:(j + 1)*B] for j in range(k)]
    # print(xbatches)
    #############################################################################
    #                             Write your code here                          #
    for i in range(k):
      # print(i)
      # print(xbatches[i].dtype)
      # print(ybatches[i].dtype)
      train(xbatches[i], ybatches[i])
    xbatches = X_train[k * B:(k + 1)*B]
    ybatches = y_train[k * B:(k + 1)*B]
    # print(xbatches.shape)
    # print(ybatches.shape)
    train(xbatches, ybatches)
    #############################################################################
    loss = mse(model(X_train), y_train)
    if (step+1) % 10 == 0:
      print(f"Loss at step {step}: {loss:.4f}")

Loss at step 9: 1.3406
Loss at step 19: 1.3405
Loss at step 29: 1.3405
Loss at step 39: 1.3404
Loss at step 49: 1.3403
Loss at step 59: 1.3403
Loss at step 69: 1.3404
Loss at step 79: 1.3403
Loss at step 89: 1.3402
Loss at step 99: 1.3403


<font color="#de3023"><h6><b>Question 7: Get predictions on your test data set and report the MSE loss on the test data.  (5 pts) </b></h6></font>

In [114]:
# Write your code here
y_pred = model(X_test)
mse(y_test, y_pred).numpy()

1.3131732